In [54]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [55]:
# open the shapefile
gdf = gpd.read_file('/app/notebooks/rapprochements/Bordeaux/20240625 - ECHANTILLON BUILDING RNB/063_3_063_8_162_Building.shp')
gdf = gdf[['BUILDINGID', 'geometry']]
gdf = gdf.to_crs(epsg=4326)
gdf['geometry'] = gdf['geometry'].force_2d()
print(gdf.head())

/venv/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(


  BUILDINGID                                           geometry
0   06311713  POLYGON ((-0.56761 44.83758, -0.5676 44.83759,...
1   06313144  POLYGON ((-0.58285 44.83633, -0.58284 44.83626...
2   06317261  POLYGON ((-0.58766 44.83882, -0.58762 44.83883...
3   06311982  POLYGON ((-0.58793 44.83795, -0.588 44.83794, ...
4   06313154  POLYGON ((-0.58609 44.83759, -0.58611 44.83759...


In [59]:
# number of buildings
print(len(gdf))

31224


In [60]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection

# sample = gdf.sample(100)

# apply a function to each row, return a dataframe
def bordeaux_rnb_intersection(row):
    cursor = connection.cursor()
    geometry = row['geometry']
    sql = f"""with areas as (
select
	rnb_id,
	ST_AREA(ST_INTERSECTION(shape,
	ST_GeomFromText('{geometry}', 4326))) as area_intersection,
	ST_AREA(shape) as area_rnb,
	ST_AREA(ST_GeomFromText('{geometry}', 4326)) as area_bordeaux
from
	batid_building
where
	ST_Intersects(shape,
	ST_GeomFromText('{geometry}', 4326)))
        select
	{row['BUILDINGID']} as BUILDINGID, rnb_id, area_intersection / NULLIF(area_rnb, 0) as rnb_inclu_bdx, area_intersection / NULLIF(area_bordeaux, 0) as bdx_inclu_rnb
from
	areas"""

    cursor.execute(sql)

    results = cursor.fetchall()

    df = None
    if len(results) == 0:
        result = [{'BUILDINGID': row['BUILDINGID'], 'rnb_id': None, 'rnb_inclu_bdx': None, 'bdx_inclu_rnb': None}]
        df = pd.DataFrame(result)
    else:
        df = pd.DataFrame(results, columns=['BUILDINGID', 'rnb_id', 'rnb_inclu_bdx', 'bdx_inclu_rnb'])
    return df
    
# apply the function to each row
gdf_compute = gdf.apply(bordeaux_rnb_intersection, axis=1)
# concatenate the results
gdf_compute = pd.concat(gdf_compute.values)

# print(sample)

/tmp/ipykernel_1599/2008446435.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf_compute = pd.concat(gdf_compute.values)


In [64]:
gdf_compute.head()

,BUILDINGID,rnb_id,rnb_inclu_bdx,bdx_inclu_rnb
0,6311713,FAEY7Q2YDWB7,0.644126,0.695464
1,6311713,5EAYR6RFTK5B,0.083437,0.233807
0,6313144,WAZVQSQDD1S8,0.682289,0.102038
1,6313144,FRQHHP6P3DVB,0.960670,0.726755
2,6313144,C6MJCAYV2Y5E,0.064633,0.048943


In [99]:
gdf_compute_2 = gdf_compute.copy()

# a match is when bothe intersection ratios are above 0.8
gdf_compute_2['match_1_1'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] > 0.8)
gdf_compute_2['rnb_inclu_bdx_b'] = (gdf_compute_2['rnb_inclu_bdx'] > 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.8)
gdf_compute_2['bdx_inclu_rnb_b'] = (gdf_compute_2['bdx_inclu_rnb'] > 0.8) & (gdf_compute_2['rnb_inclu_bdx'] <= 0.8)
gdf_compute_2['small_intersection'] = (gdf_compute_2['rnb_inclu_bdx'] <= 0.8) & (gdf_compute_2['bdx_inclu_rnb'] <= 0.8)

# print(gdf_compute_2.head())


intersections = gdf_compute_2[gdf_compute_2['rnb_id'] != None].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
match_1_1 = gdf_compute_2[gdf_compute_2['match_1_1']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
rnb_inclu_bdx = gdf_compute_2[gdf_compute_2['rnb_inclu_bdx_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
bdx_inclu_rnb = gdf_compute_2[gdf_compute_2['bdx_inclu_rnb_b']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()
small_intersection = gdf_compute_2[gdf_compute_2['small_intersection']].groupby('BUILDINGID')['rnb_id'].agg(list).reset_index()

# # merge all the dataframes
df_merge = pd.DataFrame(gdf_compute['BUILDINGID'].unique(), columns=['BUILDINGID'])
df_merge = df_merge.merge(intersections, on='BUILDINGID', how='left', suffixes=('', '_intersections'))
df_merge = df_merge.merge(match_1_1, on='BUILDINGID', how='left', suffixes=('', '_match_1_1'))
df_merge = df_merge.merge(rnb_inclu_bdx, on='BUILDINGID', how='left', suffixes=('', '_rnb_inclu_bdx'))
df_merge = df_merge.merge(bdx_inclu_rnb, on='BUILDINGID', how='left', suffixes=('', '_bdx_inclu_rnb'))
df_merge = df_merge.merge(small_intersection, on='BUILDINGID', how='left', suffixes=('', '_small_intersection'))

# clean None values in lists
df_merge['rnb_id'] = df_merge['rnb_id'].apply(lambda l: [rnb_id for rnb_id in l if rnb_id is not None])


df_merge.head(10)

,BUILDINGID,rnb_id,rnb_id_match_1_1,rnb_id_rnb_inclu_bdx,rnb_id_bdx_inclu_rnb,rnb_id_small_intersection
0,6311713,"[FAEY7Q2YDWB7, 5EAYR6RFTK5B]",NaN,NaN,NaN,"[FAEY7Q2YDWB7, 5EAYR6RFTK5B]"
1,6313144,"[WAZVQSQDD1S8, FRQHHP6P3DVB, C6MJCAYV2Y5E]",NaN,[FRQHHP6P3DVB],NaN,"[WAZVQSQDD1S8, C6MJCAYV2Y5E]"
2,6317261,"[82MB44X9FWFY, FRNFPTZD9ANP]",[82MB44X9FWFY],NaN,NaN,[FRNFPTZD9ANP]
3,6311982,[WTZBDBBQBANJ],[WTZBDBBQBANJ],NaN,NaN,NaN
4,6313154,"[QV6VTJQ4ERQ4, QV6VTJQ4ERQ4]","[QV6VTJQ4ERQ4, QV6VTJQ4ERQ4]",NaN,NaN,NaN
5,6315127,"[BQAY3YZSSZQW, 632BVJNSZ4D3, QGHPNM8G17R9, XPG...",NaN,"[BQAY3YZSSZQW, 632BVJNSZ4D3, QGHPNM8G17R9, XPG...",NaN,NaN
6,6318663,[6PTG46DNH4NF],[6PTG46DNH4NF],NaN,NaN,NaN
7,6319905,"[RA68KQH1G93D, M9Z17WTH1XW1, Q91Y3W3QWR9W, BS1...",NaN,"[RA68KQH1G93D, M9Z17WTH1XW1, Q91Y3W3QWR9W, BS1...",NaN,NaN
8,6319652,"[CBQ1DMK7F9CB, 5H7ABTQVJF1H]",NaN,[5H7ABTQVJF1H],NaN,[CBQ1DMK7F9CB]
9,6319436,[HPN29APECCVF],[HPN29APECCVF],NaN,NaN,NaN


In [102]:
# print number of rows
print(len(df_merge))

# print number of 1_1 matches
print(len(df_merge[df_merge['rnb_id_match_1_1'].notnull()]))

# print number of rnb_inclu_bdx matches
print(len(df_merge[df_merge['rnb_id_rnb_inclu_bdx'].notnull()]))

# print number of bdx_inclu_rnb matches
print(len(df_merge[df_merge['rnb_id_bdx_inclu_rnb'].notnull()]))

# number of rows with empty list of rnb_ids as intersection
print(len(df_merge[df_merge['rnb_id'].apply(lambda l: len(l) == 0)]))


31221
13607
13049
987
1884


In [100]:
# get the result for BUILDINGID 06348985
print(df_merge[df_merge['BUILDINGID'] == '06348985'])

      BUILDINGID rnb_id rnb_id_match_1_1 rnb_id_rnb_inclu_bdx  \
10828   06348985     []              NaN                  NaN   

      rnb_id_bdx_inclu_rnb rnb_id_small_intersection  
10828                  NaN                       NaN  
